In [1]:
# from a notebook or python REPL:
from rag_guard import guard_and_run
res = guard_and_run("Show me salinity profiles near the equator in March 2025", year="2025", months=["03"], top_k=30, date_from="2025-03-01", date_to="2025-03-31", out_json="../pipeline_outputs/rag_equator_mar2025.json")
print(res)

[1/6] Initializing services ...
[2/6] Semantic retrieval from Chroma ...
  Retrieved 30 semantic candidates
[3/6] Extract platform/profile candidates ...
  Extracted 30 unique platform/profile candidates
[4/6] Fetch numeric measurements from DuckDB ...
  Retrieved 64007 numeric rows from DuckDB
[5/6] Assemble context for LLM ...
[6/6] Calling LLM (Azure OpenAI) ...
LLM responded (truncated):
 In March 2025, salinity profiles near the equator show surface salinity values around 36.13–36.14 PSU at shallow pressures (2.9–21.6 dbar), with consistent readings across the sampled depths [SQL:6903139:20]. The overall mean salinity for the dataset is 35.53 PSU, with a range from 32.10 to 37.80 PSU [SQL:6903139:20]. These profiles suggest stable, high salinity in the upper ocean layers for this region and time.

Recommended visualizations:
- Plot 1: Salinity Profile — x=PRES, y=PSAL, color=platform_number
- Plot 2: Temperature vs Salinity — x=PSAL, y=TEMP, color=PRES
- Plot 3: Salinity Distribut

In [ ]:
# I want to know the highest temperature measured by US ARGO PROJECT"

import duckdb
import pandas as pd

# Connect to your DuckDB file
con = duckdb.connect("../argo.duckdb")

# Check available tables
print("Tables:", con.execute("SHOW TABLES").fetchall())

# Query: Get highest temperature measured by US ARGO PROJECT in 2025
query = """
WITH all_measurements AS (
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_01 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_02 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_03 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_04 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_05 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_06 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_07 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_08 m
    UNION ALL
    SELECT m.platform_number, m.profile_index, m.juld, m.pres, m.temp
    FROM core_measurements_2025_09 m
),
all_metadata AS (
    SELECT platform_number, project_name, pi_name
    FROM metadata_full_2025_01
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_02
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_03
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_04
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_05
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_06
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_07
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_08
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_09
)
SELECT 
    a.platform_number,
    a.profile_index,
    a.juld,
    a.pres,
    a.temp AS max_temp,
    md.project_name,
    md.pi_name
FROM all_measurements a
JOIN all_metadata md
  ON a.platform_number = md.platform_number
WHERE md.project_name ILIKE '%US ARGO%'
ORDER BY max_temp DESC
LIMIT 1;
"""

result = con.execute(query).fetchdf()
print(result)


Tables: [('calibration_2025_01',), ('calibration_2025_02',), ('calibration_2025_03',), ('calibration_2025_04',), ('calibration_2025_05',), ('calibration_2025_06',), ('calibration_2025_07',), ('calibration_2025_08',), ('calibration_2025_09',), ('core_measurements_2025_01',), ('core_measurements_2025_02',), ('core_measurements_2025_03',), ('core_measurements_2025_04',), ('core_measurements_2025_05',), ('core_measurements_2025_06',), ('core_measurements_2025_07',), ('core_measurements_2025_08',), ('core_measurements_2025_09',), ('history_2025_01',), ('history_2025_02',), ('history_2025_03',), ('history_2025_04',), ('history_2025_05',), ('history_2025_06',), ('history_2025_07',), ('history_2025_08',), ('history_2025_09',), ('metadata_clean_2025_01',), ('metadata_clean_2025_02',), ('metadata_clean_2025_03',), ('metadata_clean_2025_04',), ('metadata_clean_2025_05',), ('metadata_clean_2025_06',), ('metadata_clean_2025_07',), ('metadata_clean_2025_08',), ('metadata_clean_2025_09',), ('metadata

In [3]:
# What is the project name of the platform number 1902476

import duckdb
import pandas as pd

con = duckdb.connect("../argo.duckdb")

# Step 1: Look across all metadata_full tables for platform_number=1902476
query = """
WITH all_metadata AS (
    SELECT platform_number, project_name, pi_name
    FROM metadata_full_2025_01
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_02
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_03
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_04
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_05
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_06
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_07
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_08
    UNION
    SELECT platform_number, project_name, pi_name FROM metadata_full_2025_09
)
SELECT *
FROM all_metadata
WHERE platform_number = 1902476;
"""

result = con.execute(query).fetchdf()
print(result)


   PLATFORM_NUMBER               PROJECT_NAME  \
0          1902476  DMB, WHOI Argo equivalent   

                                             PI_NAME  
0  VIVIANE MENEZES, HEATHER FUREY, AMY BOWER, MAT...  


In [1]:
import duckdb
import pandas as pd

con = duckdb.connect("../argo.duckdb")

query = """
SELECT 
    AVG(temperature) AS avg_temperature,
    COUNT(*) AS num_records
FROM measurements_full_2025_03
WHERE latitude BETWEEN 0 AND 30
  AND longitude BETWEEN 60 AND 100
  AND temperature IS NOT NULL;
"""

result = con.execute(query).fetchdf()
print(result)


CatalogException: Catalog Error: Table with name measurements_full_2025_03 does not exist!
Did you mean "metadata_full_2025_03"?

LINE 5: FROM measurements_full_2025_03
             ^

In [ ]:
import json
p = "../pipeline_outputs/test_ui_profile.json"
d = json.load(open(p))
print("ui_payload keys:", d.get("ui_payload", {}).keys())
for item in d["ui_payload"]["ui"][:3]:
    print("TYPE:", item.get("type"), "TITLE:", item.get("title", "")[:80], "META:", item.get("meta", {}))

ui_payload keys: dict_keys([])


KeyError: 'ui_payload'